Robertas Buračevskis, 2110543, III PS, 1gr, 1užd.

Imports

In [ ]:
!pip install openimages
!pip install --upgrade pip

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.3/139.3 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 10.5 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 23.1.2
    Uninstalling pip-23.1.2:
      Successfully uninstalled pip-23.1.2


In [ ]:
import torch, torchvision
from torchvision import transforms
from torchvision.models import resnet50, ResNet50_Weights

import numpy as np
import os
from PIL import Image

from openimages.download import download_dataset
from collections import defaultdict

GPU/CPU Selection

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

print(device)

cpu


Downloading images

In [ ]:
#Nustatoma direktorija, kur bus saugomos nuotraukos bei pasirenkamos 3 klases
data_path = "openimages"
classes = ["Tank","Bee","Banana"]


if not os.path.exists(data_path):
  os.makedirs(data_path)

download_dataset(data_path, classes, limit = 350)


100%|██████████| 8/8 [00:00<00:00, 11.20it/s]


{'tank': {'images_dir': 'openimages/tank/images'},
 'bee': {'images_dir': 'openimages/bee/images'},
 'banana': {'images_dir': 'openimages/banana/images'}}

Model selection

In [ ]:
#modelio paleidimas
weights = ResNet50_Weights.DEFAULT
model = resnet50(weights=weights)
model.to(device)
model.eval()

Downloading: "https://download.pytorch.org/models/resnet50-11ad3fa6.pth" to /root/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 112MB/s]


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

Making Dataset

In [ ]:
#transformacijos normalizacija, dataset parametru nustatymas bei limitavimas, jog nuotraukos nevirsytu 1000 limito
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])
dataset = torchvision.datasets.ImageFolder("openimages/", transform = transform)
dataset_len = len(dataset)
dataset_classes = dataset.classes
if 1000 < dataset_len:
    dataset = torch.utils.data.Subset(dataset, np.random.choice(dataset_len, 1000, replace=False))

Dataloader


In [ ]:
#Klase, kurioje laikomi label'u reiksmes ir ar tai buvo ground_truth
class CustomDataset():
  def __init__(self,value:float, ground_truth: bool):
      self.value = value
      self.ground_truth = ground_truth

memo_dict = defaultdict(list)

#klasiu indexai general_classification faile - 1
index = {
    "tank": 847,
    "bee": 309,
    "banana": 954
}
#kraunama i loaderi bei ijungiamas shuffle'inimas. Sigmoido pagalba keiciamos reiksmes 0-1
loader = torch.utils.data.DataLoader(dataset,batch_size = 32, shuffle = True, num_workers = 2)
for inputs, labels in loader:
  outputs = model(inputs.to(device))
  outputs = torch.sigmoid(outputs)

#Einama per visus batchus, imam label, tada einam per klases ir atrenkam tas
# klases, kurios mums yra aktualios. Jei ground_truth, tai issaugom.
  for i in range(len(labels)):
    label = labels[i].item()
    for dclass,dclass_name in enumerate(dataset_classes):
      if(dclass_name not in index):
        continue
      label_idx = index[dclass_name]
      res = CustomDataset(value = outputs[i][label_idx].item(), ground_truth = (label == dclass))
      memo_dict[label_idx].append(res)

In [ ]:
# Saugomos ir skaiciuojamos statistikos
class Stats():
    def __init__(self):
        self.TP = 0
        self.TN = 0
        self.FP = 0
        self.FN = 0

    def accuracy(self):
        return (self.TP + self.TN) / (self.TP + self.TN + self.FP + self.FN)

    def recall(self):
        return self.TP / (self.TP + self.FN)

    def precision(self):
        return self.TP / (self.TP + self.FP)

    def f1(self):
        pres = self.precision()
        rec = self.recall()
        return 2 * (pres * rec) / (pres + rec)

    def __str__(self):
        return f"Accuracy:  {self.accuracy():}\nRecall:    {self.recall():}\nPrecision: {self.precision():}\nF1:        {self.f1():}"


# Nuskaitom klases pavadinimus is general_classification failo
with open('general_classification.txt', 'r') as class_file:
  classes = class_file.readlines()

threshold = 0.5
for label, memo in memo_dict.items():
    stats = Stats()
    for x in memo:
        above_threshold = x.value >= threshold
        stats.TP += above_threshold == True and x.ground_truth == True
        stats.TN += above_threshold == False and x.ground_truth == False
        stats.FP += above_threshold == True and x.ground_truth == False
        stats.FN += above_threshold == False and x.ground_truth == True

    print(f"{classes[label].strip()} (TP:{stats.TP} TN:{stats.TN} FP:{stats.FP} FN:{stats.FN} | Total: {stats.TP+stats.TN+stats.FP+stats.FN}):\n{str(stats)}\n")


banana (TP:330 TN:307 FP:362 FN:1 | Total: 1000):
Accuracy:  0.637
Recall:    0.9969788519637462
Precision: 0.476878612716763
F1:        0.6451612903225806

bee (TP:334 TN:389 FP:277 FN:0 | Total: 1000):
Accuracy:  0.723
Recall:    1.0
Precision: 0.546644844517185
F1:        0.7068783068783069

tank, army tank, armored combat vehicle, armoured combat vehicle (TP:334 TN:356 FP:309 FN:1 | Total: 1000):
Accuracy:  0.69
Recall:    0.9970149253731343
Precision: 0.5194401244167963
F1:        0.6830265848670757

